In [2]:
import pandas as pd
import math
import random
import numpy as np

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics 

from sklearn.naive_bayes import GaussianNB

In [3]:
fruit = pd.read_table("C:/Users/Jerry Iyi-Ewuare/Downloads/fruit_data_with_colors.txt");
fruit.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [4]:
X = fruit[["mass","width","height","color_score"]]
X.head()

,mass,width,height,color_score
0,192,8.4,7.3,0.55
1,180,8.0,6.8,0.59
2,176,7.4,7.2,0.60
3,86,6.2,4.7,0.80
4,84,6.0,4.6,0.79


In [5]:
y = fruit[["fruit_name"]]
y.head()

,fruit_name
0,apple
1,apple
2,apple
3,mandarin
4,mandarin


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4321)
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

In [7]:
for i in range(1,9):
    KNclassifier = KNeighborsClassifier(n_neighbors = i)  
    KNclassifier.fit(X_train, y_train.values.ravel())
    y_prediction = KNclassifier.predict(X_test)
    print(confusion_matrix(y_test, y_prediction))
    print("accuracy score:", metrics.accuracy_score(y_test, y_prediction))

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 0 0 6]]
accuracy score: 0.6666666666666666
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 1 0 5]]
accuracy score: 0.6111111111111112
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 0 0 6]]
accuracy score: 0.5555555555555556
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 2 0 4]]
accuracy score: 0.4444444444444444


In [8]:
Gclassifier = GaussianNB()  
Gclassifier.fit(X_train, y_train.values.ravel())
y_prediction = Gclassifier.predict(X_test)
print(confusion_matrix(y_test, y_prediction))
print("accuracy score:", metrics.accuracy_score(y_test, y_prediction))

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [2 0 0 4]]
accuracy score: 0.8888888888888888


In [228]:
def separatebyclass(X, y):
    separated = {}
    print(len(y))
    index = 0
    for rows in y.iterrows():
        label = rows['fruit_name']
        separated[label] = []
    for label in separated:
        separated[label].append(X[index])
        index = index + 1
    return separated        

In [142]:
def countOccurance(y):
    #occurance = y["fruit_name"].value_counts()
    labels = y['fruit_name'].value_counts().keys().tolist()
    counts = y['fruit_name'].value_counts().tolist()
    
    occurance = {}
    index = 0
    for i in labels:
        if i not in occurance:
            occurance[i] = 0
        occurance[i] = counts[index]
        index = index + 1
    return occurance

In [50]:
def mean(values):
    return sum(values)/float(len(values))

In [67]:
def stdev(values):
    average = mean(values)
    variance = (sum([pow(x - average, 2) for x in values]))/float(len(values)-1)
    return math.sqrt(variance)

In [111]:
def model(X):
    result = [(mean(i), stdev(i)) for i in X]
    return result

In [169]:
def modelbyclass(X, y):
    separated = separatebyclass(X, y)
    print(separated)
    classModels = {}
    for labels in separated:
        classModels[labels] = model(separated[labels])
    return classModels

In [109]:
def calculatePdf(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean, 2)/(2 * math.pow(stdev, 2))))
    return (1/(math.sqrt(2 * math.pi) * stdev))*exponent

In [187]:
def calclassprob(models, i):
    probabilities = {}
    occurance = countOccurance(y_train)
    index = 0
    for labels in models:
        probabilities[labels] = 1
    for labels in models:
        values = models[labels]
        probabilities[labels]  *= calculatePdf(i[index], values[0][0], values[0][1])
        index = index + 1
    for label in probabilities:
        for keys in occurance:
            if labels == keys:
                probabilities[label] *= (occurance[keys]/41)
    return probabilities

In [172]:
def predict(models, i):
    probabilities = calclassprob(models, i)
    bestLabel, bestProb = None, -1
    for labels in probabilities:
        if bestLabel is None or probability > bestProb:
            bestProb = probabilities[labels]
            bestLabel = labels
    return bestLabel

In [183]:
def getPredictions(models, X):
    predictions = []
    for i in X:
        result = predict(models, i)
        predictions.append(result)
    return predictions

In [14]:
# models = modelbyclass(X_train, y_train)
# predictions = getPredictions(models, X_test)
# print(predictions)
print(confusion_matrix(y_test, predictions))
print("accuracy score:", metrics.accuracy_score(y_test, predictions))
print()
print(y["fruit_name"].value_counts())
print(y_train["fruit_name"].value_counts())

labels = y['fruit_name'].value_counts().keys().tolist()
print(labels)
counts = y['fruit_name'].value_counts().tolist()
print(counts)


orange      19
apple       19
lemon       16
mandarin     5
Name: fruit_name, dtype: int64
lemon       14
orange      13
apple       12
mandarin     2
Name: fruit_name, dtype: int64
['orange', 'apple', 'lemon', 'mandarin']
[19, 19, 16, 5]
